## Dependencies



In [1]:
!pip install mftool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [2]:
from mftool import Mftool
import pandas as pd
import datetime

## Select Mutual Fund House and Scheme

In [3]:
mf = Mftool()

In [4]:
# get all supported amc details
amc_details = mf.get_all_amc_profiles(as_json=False)
# amc_details

In [5]:
df_amcs = pd.DataFrame(amc_details)
# df_amcs

In [6]:
# let's select a reputed fund house
df_amc = df_amcs[df_amcs["Name of the Mutual Fund"] == "PPFAS Mutual Fund"]
df_amc

,Name of the Mutual Fund,Date of set up of Mutual Fund,Name(s) of Sponsor,Name of Trustee Company,Name of Trustees,Name of Assest Management Co.,Date of Incorporation of AMC,Name(s) of Director,Name of Chairman,Name of Chief Executive Officer,...,Name(s) of Company Secretary,Name(s) of Fund Manager,Name of Compliance Officer,Name of Chief Bussiness Officer,Name of the Chief Investment Officer,Name(s) of the Chief Investment Officer - Dept,Name of Head of Operations,Name(s) of the Chief Investment Officer - Equity,Name of President,Name of Wholetime Director
31,PPFAS Mutual Fund,"October 10, 2012",Parag Parikh Financial Advisory Services Ltd.,PPFAS Trustee Company Private Limited,Bhagirat Merchant - Independent Director ...,PPFAS Asset Management Private Limited,"August 08, 2011",Neil Parag Parikh ...,NaN,Neil Parag Parikh,...,NaN,Raj Mehta ...,Priya Hariani,NaN,Rajeev Thakkar,NaN,Jignesh Desai ...,NaN,NaN,Rajeev Thakkar


In [7]:
# what is the fund house ID?
df_amc.columns.to_list()

['Name of the Mutual Fund',
 'Date of set up of Mutual Fund',
 'Name(s) of Sponsor',
 'Name of Trustee Company',
 'Name of Trustees',
 'Name of Assest Management Co.',
 'Date of Incorporation of AMC',
 'Name(s) of Director',
 'Name of Chairman',
 'Name of Chief Executive Officer',
 'Name of Managing Director',
 'Name of Compliance Officer & Company Secretary',
 'Name of Investor Service Officer',
 'Address of AMC',
 'Telephone Number',
 'Fax Number',
 'Website',
 'Email',
 'Name(s) of Auditors ',
 'Name(s) of Custodian ',
 'Name(s) of Registrar and Transfer Agent',
 'Name of Head Equity',
 'Name of Head-Fixed Income',
 'Name of Sales Head',
 'Name(s) of the Chief Operating Officer',
 'Name(s) of Company Secretary',
 'Name(s) of Fund Manager',
 'Name of Compliance Officer',
 'Name of Chief Bussiness Officer',
 'Name of the Chief Investment Officer',
 'Name(s) of the Chief Investment Officer - Dept',
 'Name of Head of Operations',
 'Name(s) of the Chief Investment Officer - Equity',
 'Na

In [8]:
# no ID here
# let's fetch funds now

df_schemes = pd.Series(mf.get_scheme_codes())
# df_schemes

In [9]:
# filter the funds from selected fund house
df_schemes[df_schemes.str.contains("Parag Parikh")]

143263    Parag Parikh Liquid Fund- Direct Plan- Daily R...
143269        Parag Parikh Liquid Fund- Direct Plan- Growth
143262    Parag Parikh Liquid Fund- Direct Plan- Monthly...
143265    Parag Parikh Liquid Fund- Direct Plan- Weekly ...
143264    Parag Parikh Liquid Fund- Regular Plan- Daily ...
143260       Parag Parikh Liquid Fund- Regular Plan- Growth
143261    Parag Parikh Liquid Fund- Regular Plan- Monthl...
143266    Parag Parikh Liquid Fund- Regular Plan- Weekly...
147481           Parag Parikh Tax Saver Fund- Direct Growth
147482          Parag Parikh Tax Saver Fund- Regular Growth
122639    Parag Parikh Flexi Cap Fund - Direct Plan - Gr...
122640    Parag Parikh Flexi Cap Fund - Regular Plan - G...
148958    Parag Parikh Conservative Hybrid Fund - Direct...
148961    Parag Parikh Conservative Hybrid Fund - Direct...
148959    Parag Parikh Conservative Hybrid Fund - Regula...
148960    Parag Parikh Conservative Hybrid Fund - Regula...
dtype: object

In [10]:
# let's select the fund category
df_fund = df_schemes[(df_schemes.str.contains("Parag Parikh")) & (df_schemes.str.contains("Flexi")) & (df_schemes.str.contains("Direct"))]
df_fund

122639    Parag Parikh Flexi Cap Fund - Direct Plan - Gr...
dtype: object

## Collect Historic Data & Make Ready To Access

In [11]:
df_nav = mf.get_scheme_historical_nav(df_fund.index.item(), as_Dataframe=True)
df_nav

,nav,dayChange
date,,
04-11-2022,50.96810,-0.3444
03-11-2022,51.31250,-0.3929
02-11-2022,51.70540,-0.2795
01-11-2022,51.98490,0.0385
31-10-2022,51.94640,0.4248
...,...,...
03-06-2013,10.05720,0.0418
31-05-2013,10.01540,-0.0173
30-05-2013,10.03270,0.0247


In [12]:
# df_nav.dtypes

In [13]:
# # cast nav to float64

# df_nav["nav"] = df_nav["nav"].astype("float64")
# df_nav.dtypes

In [14]:
# df_nav.index.dtype

In [15]:
# df_nav.index[0]

In [16]:
# # done below
# # cast index to datetime

# df_nav.index = pd.to_datetime(df_nav.index, infer_datetime_format=True, utc=True)
# df_nav.index

In [17]:
# ways to access the datetimeindex

# print(df_nav.loc["2013-05-28"])
# print(df_nav.loc[datetime.datetime.strptime("2013-05-28", "%Y-%m-%d").replace(tzinfo=datetime.timezone.utc)])

In [18]:
# one time check done

# check for saturday and sunday nav

today = datetime.datetime.now().replace(tzinfo=datetime.timezone.utc)
sunday_idx = (today.weekday() + 1) % 7

sun = today - datetime.timedelta(days=sunday_idx, hours=today.hour, minutes=today.minute, seconds=today.second, microseconds=today.microsecond)
sun

datetime.datetime(2022, 11, 6, 0, 0, tzinfo=datetime.timezone.utc)

In [19]:
# one time check done

# try:
#   df_nav.loc[sun]
# except:
#   print("Data missing for Sunday. As expected!")

In [20]:
# way to access individual attribute

# df_nav.index.month

In [21]:
# way to search in datetimeindex

# "2013-05-28" in df_nav.index

In [22]:
# data will always be missing for holidays, and weekends
# on holidays, the nav stays the same as previous day's nav

# refer: https://pandas.pydata.org/pandas-docs/stable/development/extending.html

@pd.api.extensions.register_dataframe_accessor("safe_nav")
class NAVAccessor:
    def __init__(self, pandas_obj):
        self._validate(pandas_obj)
        pandas_obj = self._preprocess(pandas_obj)
        self.start_date = pandas_obj.index[0]
        self._obj = pandas_obj

    @staticmethod
    def _validate(obj):
        # verify there is a column nav and a column dayChange,
        # and index must be datetime
        if "nav" not in obj.columns or "dayChange" not in obj.columns:
        # or not pd.api.types.is_datetime64_any_dtype(obj.index):
            raise AttributeError("Must have 'nav' and 'dayChange'") #. Index must be 'datetimeindex'.")

    @staticmethod
    def _preprocess(obj):
        # search, sort, type conversions, etc.
        
        # cast to datetimeindex
        obj.index = pd.to_datetime(obj.index, format="%d-%m-%Y", utc=True)

        # other conversions
        obj["nav"] = obj["nav"].astype("float64")

        # # sort ascending by date
        # # assumption: data is already sorted in descending order
        # obj = obj[::-1]

        return obj

    def is_date_available(self, date):
      return date in self._obj.index

    # let's create a separate function which handles this
    def for_date(self, date):
      if date > self.start_date:
        # go back by a day till holidays are not over 
        while not self.is_date_available(date):
          date -= datetime.timedelta(days=1)
        
        # return the data of the day before holidays
        return self._obj.loc[date]

      # return the data of inception if data is from very past
      return self._obj.loc[self.start_date]

In [23]:
df_nav.safe_nav._obj.index

DatetimeIndex(['2022-11-04 00:00:00+00:00', '2022-11-03 00:00:00+00:00',
               '2022-11-02 00:00:00+00:00', '2022-11-01 00:00:00+00:00',
               '2022-10-31 00:00:00+00:00', '2022-10-28 00:00:00+00:00',
               '2022-10-27 00:00:00+00:00', '2022-10-25 00:00:00+00:00',
               '2022-10-21 00:00:00+00:00', '2022-10-20 00:00:00+00:00',
               ...
               '2013-06-10 00:00:00+00:00', '2013-06-07 00:00:00+00:00',
               '2013-06-06 00:00:00+00:00', '2013-06-05 00:00:00+00:00',
               '2013-06-04 00:00:00+00:00', '2013-06-03 00:00:00+00:00',
               '2013-05-31 00:00:00+00:00', '2013-05-30 00:00:00+00:00',
               '2013-05-29 00:00:00+00:00', '2013-05-28 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='date', length=2323, freq=None)

In [24]:
# sort df_nav
df_nav = df_nav[::-1]
df_nav.index

DatetimeIndex(['2013-05-28 00:00:00+00:00', '2013-05-29 00:00:00+00:00',
               '2013-05-30 00:00:00+00:00', '2013-05-31 00:00:00+00:00',
               '2013-06-03 00:00:00+00:00', '2013-06-04 00:00:00+00:00',
               '2013-06-05 00:00:00+00:00', '2013-06-06 00:00:00+00:00',
               '2013-06-07 00:00:00+00:00', '2013-06-10 00:00:00+00:00',
               ...
               '2022-10-20 00:00:00+00:00', '2022-10-21 00:00:00+00:00',
               '2022-10-25 00:00:00+00:00', '2022-10-27 00:00:00+00:00',
               '2022-10-28 00:00:00+00:00', '2022-10-31 00:00:00+00:00',
               '2022-11-01 00:00:00+00:00', '2022-11-02 00:00:00+00:00',
               '2022-11-03 00:00:00+00:00', '2022-11-04 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='date', length=2323, freq=None)

In [25]:
df_nav.safe_nav._obj.index[0].day, df_nav.safe_nav._obj.index[0].month

(28, 5)

In [26]:
# check if it works
df_nav.safe_nav.for_date(sun).name.day

4

In [27]:
def get_date(year, month, day):
  return datetime.datetime(year=2022, month=11, day=7, tzinfo=datetime.timezone.utc)

In [28]:
def get_yesterday():
  today = datetime.datetime.now()
  yesterday = today - datetime.timedelta(days=1)
  yesterday = get_date(yesterday.year, yesterday.month, yesterday.day)
  return yesterday

## Calculate Returns

### Absolute Returns

In [29]:
def one_day_absolute_return(df_nav):
  yesterday = get_yesterday()

  day_before_yesterday = yesterday - datetime.timedelta(days=1)

  return df_nav.safe_nav.for_date(yesterday)["dayChange"]*100 / df_nav.safe_nav.for_date(day_before_yesterday)["nav"]

one_day_absolute_return(df_nav)

-0.6757167718631856

In [30]:
def absolute_return(initial_value, final_value):
  return (final_value - initial_value) * 100 / initial_value

In [31]:
def one_year_absolute_return(df_nav):
  yesterday = get_yesterday()

  one_year_before_yesterday = (yesterday - datetime.timedelta(days=365))

  return absolute_return(df_nav.safe_nav.for_date(one_year_before_yesterday)["nav"], df_nav.safe_nav.for_date(yesterday)["nav"])

one_year_absolute_return(df_nav)

-4.817920370508706

In [33]:
df_nav.index[-1]

Timestamp('2022-11-04 00:00:00+0000', tz='UTC')

In [34]:
def year_to_date_absolute_return(df_nav):
  yesterday = get_yesterday()
  
  year_start = datetime.datetime(year=datetime.datetime.now().year, month=1, day=1, tzinfo=datetime.timezone.utc)

  # handling holidays at the start of the year
  while not df_nav.safe_nav.is_date_available(year_start):
    year_start += datetime.timedelta(days=1)

  return (df_nav.safe_nav.for_date(yesterday)["nav"] - df_nav.safe_nav.for_date(year_start)["nav"])*100 / df_nav.safe_nav.for_date(year_start)["nav"]

year_to_date_absolute_return(df_nav)

-6.631995075876927

In [35]:
# generic function

def n_years_absolute_return(df_nav, n_years=7):
  yesterday = get_yesterday()
  n_years_before_yesterday = (yesterday - datetime.timedelta(days=365*n_years))

  return (df_nav.safe_nav.for_date(yesterday)["nav"] - df_nav.safe_nav.for_date(n_years_before_yesterday)["nav"])*100 / df_nav.safe_nav.for_date(n_years_before_yesterday)["nav"]

In [36]:
for n_years in (1, 3, 5, 7):
  return_ = n_years_absolute_return(df_nav, n_years=n_years)
  print("Years:", n_years, "\t", "Abs. Return:", return_)

Years: 1 	 Abs. Return: -4.817920370508706
Years: 3 	 Abs. Return: 85.71673225477335
Years: 5 	 Abs. Return: 120.00492083498801
Years: 7 	 Abs. Return: 196.4778491321141


### Trailing Returns (AKA. CAGR i.e. Compounded Annual Growth Rate)

**Understanding**

Trailing Return cancels the compounding effect from the absolute return.

If it is a year-on-year compounding, trailing return tells you the yearly simple interest rate that would have applied to compound and reach the final amount in the given time span.

For example, the absolute return for 7 years, which is 196.37% does not consider the time that you'll have to wait to earn this much profit.

Like if we want to compare the return of this mutual fund with a typical compound interest scheme, we want to know the yearly simple interest rate that applies. What if a compound interest scheme for 7 years would generate more than 196.37% absolute return? You'd never know.

Trailing return helps us with calculating the yearly rate of interest that you'd need to reach upto 196.37% in 7 years. Basically, it remove the timefactor from the absolute return and gets you the yearly interest rate.

In [28]:
# formulation

# compound interest formula

# final_value = initial_value * (1 + (interest_rate / num_times_interest_applied_per_period) ^ (num_times_interest_applied_per_period*num_periods))

# for 5 year compounding and interest applied yearly
# num_times_interest_applied_per_period = 1, num_periods=5

# for 5 year compounding and interest applied mothly
# num_times_interest_applied_per_period = 12, num_periods=5


# deriving the formula of trailing returns from above, we get

# trailing_return = ((final_value / initial_value) ^ (1 / num_times_interest_applied_per_period*num_periods) - 1) * num_times_interest_applied_per_period)

In [37]:
# for most of the compounding schemes, num_times_interest_applied_per_period = 1

def n_years_trailing_return(df_nav, n_years):
  yesterday = get_yesterday()
  n_years_before_yesterday = (yesterday - datetime.timedelta(days=365*n_years))

  final_value = df_nav.safe_nav.for_date(yesterday)["nav"]
  initial_value = df_nav.safe_nav.for_date(n_years_before_yesterday)["nav"]

  return (((final_value / initial_value) ** (1 / n_years)) - 1)*100

In [38]:
for n_years in (1, 3, 5, 7):
  return_ = n_years_trailing_return(df_nav, n_years=n_years)
  print("Years:", n_years, "\t", "Trl. Return:", return_)

Years: 1 	 Trl. Return: -4.817920370508711
Years: 3 	 Trl. Return: 22.918431885817746
Years: 5 	 Trl. Return: 17.081015049783232
Years: 7 	 Trl. Return: 16.79586420189654


**Output for future**

```
Years: 1 	 Abs. Return: -4.817920370508706
Years: 3 	 Abs. Return: 85.65517191136884
Years: 5 	 Abs. Return: 119.47250570555055
Years: 7 	 Abs. Return: 196.37096305255452
```

```
Years: 1 	 Trl. Return: -4.817920370508711
Years: 3 	 Trl. Return: 22.904848946865954
Years: 5 	 Trl. Return: 17.024292561361843
Years: 7 	 Trl. Return: 16.789847958013524
```

**Observations:**
1. The fund has performed significantly well over the past 3 years, however, it's performance dropped last year. Basically, the fund performed very well from November 2019 to November 2021, and it's performance dropped after November 2021 till November 2022.
2. We can confirm the same by looking at absolute returns. Out of 119.47% over 5 years, 85.65% alone was generate in the last 3 years.
3. The fund is not able to maintain it's yearly interest rate i.e. trailing return value in the last year. If you had to invest in this fund, ignoring the last year, you could expect an average annual return of 16% or more in this fund, given that you stay invested for more than 3 years.

**NOTE:** For 1 year, trailing return = absolute return as there is no compounding effect

### Rolling Returns

**Understanding**

Trailing Return gives you an estimate about the year-on-year performance of the fund, if invested for a long term.

Now, let's say for example that you have two funds having the same CAGR of 15% over the period of 5 years. How would you select one of them?

Or let's say we consider absolute returns, and both the funds have same return value of 100% over the period of 5 years. Again, how would you select a fund of these two?

What a professional would do is check for consistency.

For example, if fund A had +300% in the first 2 years, -100% in the 3rd year, +100% in the 4th year and -200% in the last year, and fund B had +50% in the first 2 years, -10% in the 3rd year, +20% in the 4th year and +40% in the fifth year, which one would you select?

I'd have opted for Fund B, as it is more stable and hence, less risky.

Rolling returns give you a way to check on the stability of returns by calculating the returns over a certain time period.

In the above case, we would select a 5 year time span and calculate different values of returns
- jan 2017 - jan 2018
- feb 2017 - feb 2018
- mar 2017 - mar 2018
- .
- .
- .
- dec 2020 - dec 2021
- jan 2021 - jan 2022


By having a look at these values, we will be able to understand how volatile/stable a fund is over the period of 5 years. The above values are calculated using 1 year data, hence they are called "rolling 1-year returns for a period of 5 years".

In [39]:
df_nav

,nav,dayChange
date,,
2013-05-28 00:00:00+00:00,9.9992,NaN
2013-05-29 00:00:00+00:00,10.0080,0.0088
2013-05-30 00:00:00+00:00,10.0327,0.0247
2013-05-31 00:00:00+00:00,10.0154,-0.0173
2013-06-03 00:00:00+00:00,10.0572,0.0418
...,...,...
2022-10-31 00:00:00+00:00,51.9464,0.4248
2022-11-01 00:00:00+00:00,51.9849,0.0385
2022-11-02 00:00:00+00:00,51.7054,-0.2795


In [40]:
# rolling returns over 5 year time span on a monthly basis
df_nav["nav"].rolling(365*5, closed="both").apply(lambda x: absolute_return(x.iloc[0], x.iloc[-1])).dropna()[::30]

date
2020-10-28 00:00:00+00:00    229.343347
2020-12-11 00:00:00+00:00    263.226753
2021-01-25 00:00:00+00:00    291.480866
2021-03-09 00:00:00+00:00    285.547260
2021-04-27 00:00:00+00:00    282.437033
2021-06-09 00:00:00+00:00    295.856433
2021-07-22 00:00:00+00:00    347.012985
2021-09-03 00:00:00+00:00    330.642390
2021-10-19 00:00:00+00:00    326.438323
2021-12-03 00:00:00+00:00    277.889376
2022-01-14 00:00:00+00:00    283.357544
2022-02-28 00:00:00+00:00    241.070993
2022-04-13 00:00:00+00:00    244.654482
2022-05-30 00:00:00+00:00    203.480677
2022-07-11 00:00:00+00:00    203.687845
2022-08-24 00:00:00+00:00    213.405936
2022-10-07 00:00:00+00:00    203.834941
Name: nav, dtype: float64

In [41]:
# having a look at the above, it seems like the missing dates are causing issues

df_rr_temp = df_nav["nav"].rolling(365*5, closed="both").apply(lambda x: absolute_return(x.iloc[0], x.iloc[-1])).dropna()
df_rr_temp.groupby([df_rr_temp.index.year, df_rr_temp.index.month]).first()

date  date
2020  10      229.343347
      11      223.336689
      12      264.029197
2021  1       272.589714
      2       290.988640
      3       288.613027
      4       282.199661
      5       291.710946
      6       294.743545
      7       316.695727
      8       339.457820
      9       333.976606
      10      318.451892
      11      299.578808
      12      279.207291
2022  1       273.721648
      2       267.496191
      3       237.565468
      4       250.457457
      5       215.633134
      6       201.490579
      7       188.307505
      8       202.738981
      9       206.373022
      10      197.192762
      11      213.819936
Name: nav, dtype: float64

In [42]:
df_rr_temp.groupby([df_rr_temp.index.year, df_rr_temp.index.month]).last()

date  date
2020  10      225.738409
      11      262.256205
      12      271.648662
2021  1       283.905749
      2       285.969350
      3       278.649799
      4       289.454252
      5       296.878502
      6       312.579515
      7       348.108853
      8       333.393185
      9       323.650311
      10      291.209099
      11      279.675418
      12      270.644806
2022  1       261.441171
      2       241.070993
      3       246.371659
      4       222.141278
      5       204.804605
      6       188.035676
      7       195.195596
      8       209.110783
      9       197.432963
      10      211.894855
      11      209.287465
Name: nav, dtype: float64

In [ ]:
# todo: take daywise and then average of a month